In [2]:
%cd ..

/Users/datnt/Desktop/code/text-normalization


# Basic statistics

In [19]:
import os
import pandas as pd

file = "data_storage/processed/retagged_data_cleaned.csv"

df = pd.read_csv(file)

# df.head(10)

In [20]:
len(df)

303916

In [24]:
for col in df.columns:
  print(col)

input
s_output
tagged_sentence
tags
gpt_tagged_sentence
gpt_tags
final_tags


In [22]:
tag_counts = (
    df["final_tags"]
    .dropna()
    .str.split(";")
    .explode()
    .str.strip()
    .value_counts()
)

print(tag_counts)

final_tags
NUM_INT           218744
DM                 51482
MEASURE            43724
NUM_INT1           25529
NUM_FLOAT          18208
RANGE              16482
DMY                11475
MY                 10062
SCORE               6645
HM                  5552
MEASURE_RANGE       4477
CURRENCY            4046
FRACTION            3475
TEL                 2083
QY                  1264
ROMAN               1142
YY                   895
DD                   783
MATH_OPERATOR        696
DDM                  655
DMDM                 412
CURRENCY_RANGE       410
HH                   371
DMYDMY               339
HOUR                 228
DDMY                 202
MYMY                 182
RATIO                127
DMDMY                 97
HMHM                  70
MS                    67
MMY                   57
MM                    57
HMS                   16
ROMAN_RANGE            9
SS                     8
T_MM                   2
Name: count, dtype: int64


In [23]:
from core.config.config import REGEX_RULE_LIST


for category in REGEX_RULE_LIST:
    tag = category["name"]
    count = tag_counts.get(tag, 0)
    if count < 70:
        print(f"{tag}: {count} sentences")

MMY: 57 sentences
QQY: 0 sentences
QQ: 0 sentences
MM: 57 sentences
HMSHMS: 0 sentences
MSMS: 0 sentences
HMS: 16 sentences
T_MM: 2 sentences
SS: 8 sentences
MS: 67 sentences
TEL1: 0 sentences
TEL2: 0 sentences
TEL3: 0 sentences
TEL4: 0 sentences
HOUR_MEASURE: 0 sentences
NUM:NUM: 0 sentences
NUM-NUM: 0 sentences
NUM/NUM: 0 sentences
NUM.NUM: 0 sentences
ROMAN_RANGE: 9 sentences


In [ ]:
# def clean_tags_field(tag_str):
#     tag_set = set()
#     if pd.notna(tag_str):
#         for tag in tag_str.split(";"):
#             tag = tag.strip()
#             if tag:
#                 tag_set.add(tag)
#     return "; ".join(sorted(tag_set))

# def clean_tags_row(row):
#     row["tags"] = clean_tags_field(row["tags"])
#     row["gpt_tags"] = clean_tags_field(row["gpt_tags"])
#     row["final_tags"] = clean_tags_field(row["final_tags"])
#     return row

# df = df.apply(clean_tags_row, axis=1)

# # Optional: save cleaned file
# df.to_csv("/Users/datnt/Desktop/code/text-normalization/data_storage/processed/true_tags_data_cleaned.csv", index=False)